In [1]:
import asyncio
import websockets
import json
import pandas as pd
import datetime as dt
import numpy as np

import nest_asyncio
nest_asyncio.apply()

from tardis_dev import datasets

import pandas as pd


async def call_api(msg):
    async with websockets.connect('wss://test.deribit.com/ws/api/v2') as websocket:
       await websocket.send(msg)
       while websocket.open:
           response = await websocket.recv()
           return response



def async_loop(api, message):
    return asyncio.get_event_loop().run_until_complete(api(message))


def retrieve_historic_data(start, end, instrument, timeframe):
    msg = \
        {
            "jsonrpc": "2.0",
            "id": 833,
            "method": "public/get_tradingview_chart_data",
            "params": {
                "instrument_name": instrument,
                "start_timestamp": start,
                "end_timestamp": end,
                "resolution": timeframe
            }
        }
    resp = async_loop(call_api, json.dumps(msg))

    return resp

def json_to_dataframe(json_resp):
    res = json.loads(json_resp)

    df = pd.DataFrame(res['result'])

    df['ticks'] = df.ticks / 1000
    df['timestamp'] = [dt.datetime.fromtimestamp(date) for date in df.ticks]

    return df


def get_instruments(kind = 'option'):
    msg = \
        {
            "jsonrpc": "2.0",
            "id": 833,
            "method": "public/get_instruments",
            "params": {
                "currency": "BTC",
                "kind": kind,
                "expired": False
            }
        }
    resp = async_loop(call_api, json.dumps(msg))

    res = json.loads(resp)
    df = pd.DataFrame(res['result'])

    instruments = df.instrument_name.values

    return instruments

In [2]:
start_date, end_date = \
    pd.to_datetime(["2022-08-21", "2022-09-20"]).values.astype(int) // 1000000
start_date = int(start_date)
end_date = int(end_date)

timeframe = '60'

instruments = get_instruments()

In [3]:
underlyings = get_instruments('future')
underlyings = [el for el in underlyings if el != 'BTC-PERPETUAL']

In [ ]:
dfs = {}

for instrument in instruments:

    json_resp = retrieve_historic_data(start_date, end_date, instrument, timeframe)

    df = json_to_dataframe(json_resp)
    
    if len(df):
        dfs[instrument] = df

In [ ]:
dfs_ = {}

for key, df in dfs.items():
    cur, exp, strike, typ = key.split('-')
    strike = int(strike)
    
    new_key = '-'.join([cur, exp, typ])
    
    df['strike'] = strike * np.ones((len(df), ))
    
    if new_key not in dfs_:
        dfs_[new_key] = pd.DataFrame()
    dfs_[new_key] = \
        pd.concat(
            [dfs_[new_key], 
             df
            ]
        ).reset_index(drop=True)

In [ ]:
underlying_prices = {}

for instrument in underlyings:
    json_resp = retrieve_historic_data(start_date, end_date, instrument, timeframe)
    df = json_to_dataframe(json_resp)
    underlying_prices[instrument] = df

In [ ]:
merged_datasets = {}

for instrument in underlyings:
    lft = dfs_[instrument + '-C']
    rght = underlying_prices[instrument]
    merged_datasets[instrument + '-C'] = \
        pd.merge(lft, rght,  on='timestamp', how='left', suffixes=('_option', '_underlying'))    
    lft = dfs_[instrument + '-P']
    merged_datasets[instrument + '-P'] = \
        pd.merge(lft, rght,  on='timestamp', how='left', suffixes=('_option', '_underlying'))